# 0.0 IMPORTS

In [ ]:
import os

import math
import numpy  as np
import pandas as pd
import inflection

import seaborn as sns

from matplotlib import pyplot as plt
from IPython.core.display import HTML

## 0.1. Helper Functions 

In [ ]:
%matplotlib inline
    %pylab inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set()
    
    
#funçao p/ aumentar tamanho dos graficos

## 0.2. Loanding data 

In [ ]:
os.getcwd()
#usar para saber em que diretório estou

In [ ]:
df_sales_raw = pd.read_csv ( "/home/rayane/repos/DataScience_Em_Producao/data/train.csv", low_memory = False)
df_store_raw = pd.read_csv ( "/home/rayane/repos/DataScience_Em_Producao/data/store.csv", low_memory = False)

#merge (unindo utilizando pandas)

df_raw = pd.merge(df_sales_raw, df_store_raw, how="left", on="Store")


In [ ]:
df_raw.sample()

# 1.0. DESCRICAO DOS DADOS

## 1.1. Rename Columns 

In [ ]:
df1 = df_raw.copy() #sempre que mudar de secao o ideal é copiar o dataset que eu modifiquei

In [ ]:
df_raw.columns #saber o nome das colunas

In [ ]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval']

#mudando o estilo de como ta escrevendo

snakecase = lambda x: inflection.underscore( x ) #criei uma funcao usando o lambda pra mudar o estilo de escrita
cols_new = list( map( snakecase, cols_old) ) #mapiei os nomes em formato de lista

#rename 

df1.columns = cols_new
df1.columns #mudou

In [ ]:
df1.sample() #pra saber como ficou

## 1.2. Data Dimensions 

qual é a quantidade de linhas e colunas do meu dataset

In [ ]:
print ("Number of Rows: {}".format (df1.shape[0])) #sabendo o tamanho das linhas
print ("Number of columns: {}".format (df1.shape[1])) #sabendo o tamanho da coluna

## 1.3 Data Types 
quais são os tipos de dados?


In [ ]:
df1.dtypes
#observe que date pex. esta como object (tudo no pandas que nao e int sera objeto = string). Como mudar?



In [ ]:
df1["date"] = pd.to_datetime( df1["date"]) #alterando o tipo de variavel
df1.dtypes

## 1.4 Check NA

In [ ]:
df1.isna().sum() #mostraa soma de NAS em cada coluna (a soma e mostrada com sum. isna mostra se tem pelo menos 1



## 1.5. Fillout NA
primeiro vou ver as colunas que tem NA e vou aplicar técnicas de negócios a elas.

In [ ]:
#saber a distancia max
df1.competition_distance.max()

In [ ]:
#competition_distance ~dist de uma loja competidora. 0 = muito longe ou inexistente. Sol = trocar 0 por num acima da dist max
df1["competition_distance"] = df1["competition_distance"].apply(lambda x: 200000.0 if math.isnan( x ) else x )

#competition_open_since_month ~mes em q loja competidora abriu. na = n tem ou n sabe. So. = colocar o mes da dta de venda  
df1['competition_open_since_month'] = df1.apply( lambda x: x['date'].month if  
                                                math.isnan( x['competition_open_since_month'] )
                                                else x['competition_open_since_month'], axis=1 )

#competition_open_since_year ~ano em q loja competidora abriu. na = n tem ou n sabe. Sol. = colocar o ano da dta de   

df1["competition_open_since_year"] = df1.apply(lambda x: x["date"].year if
                                               math.isnan( x["competition_open_since_year"] )
                                               else x["competition_open_since_year"], axis=1)
                             
#promo2_since_week ~cont da prom padrao. Algumas lojas entram e outras nao. na=nao participou. sol. substituir pela data
df1["promo2_since_week"] = df1.apply(lambda x: x["date"].week if
                                               math.isnan( x["promo2_since_week"] )
                                               else x["promo2_since_week"], axis=1)

#promo2_since_year ~cont da prom padrao. Algumas lojas entram e outras na. na=nao participou. Sol. substituir pela data
df1["promo2_since_year"] = df1.apply(lambda x: x["date"].year if
                                               math.isnan( x["promo2_since_year"] )
                                               else x["promo2_since_year"], axis=1)
               
month_map = {1: "Jan", 2: "Fev", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun", 
 7: "Jul", 8: "Aug", 9: "Set", 10: "Oct", 11: "Nov", 12: "Dec"}

df1["promo_interval"].fillna(0, inplace=True)

df1["month_map"] = df1["date"].dt.month.map(month_map)

df1['is_promo'] = df1[['promo_interval', 'month_map']].apply( lambda x: 0 if x['promo_interval'] == 0 
                                                             else 1 if x['month_map'] 
                                                             in x['promo_interval'].split( ',' ) 
                                                             else 0, axis=1 )


In [ ]:
df1.isna().sum()

## 1.6. Change Types
importante checar se as operações não alteraram os tipos das variaveis 

In [ ]:
df1.dtypes


In [ ]:
df1['competition_open_since_month'] =  df1["competition_open_since_month"].astype(int)
df1['competition_open_since_year'] =  df1["competition_open_since_year"].astype(int)

df1["promo2_since_week"] =  df1["promo2_since_week"].astype(int)
df1["promo2_since_year"] =  df1["promo2_since_year"].astype(int)

In [ ]:
df1.dtypes

## 1.7. Descriptive Statistical  
 serve para ganhar conhecimento de negocio e ajuda na deteccao de alguns erros
 
 ~dispersão = saber o quao esparcos meus dados estao de acordo com a tendencia central (media ou mediana) e tendencia central = resumem os dados num unico numero 
 
 curtose - concentracao dos dados. + = afunilada - = grossa
 
 skew - - concentração na esquerda e + a concentração é na direita

In [ ]:
#primeiro vou separar o df em numerica e escrita
num_attributes = df1.select_dtypes( include = ['int64', 'float64']) #numeric
cat_attributes = df1.select_dtypes( exclude = ['datetime64[ns]','int64', 'float64' ]) #categoric

In [ ]:
num_attributes.sample() #numerico

In [ ]:
cat_attributes.sample() #categoricos

### 1.7.1 Numerical Attributes

In [ ]:
#central tendency - mean, median
ct1 =  pd.DataFrame( num_attributes.apply( np.mean)).T
ct2 =  pd.DataFrame( num_attributes.apply( np.median)).T


#dispersion - std, min, max, range, skew, kurtosis
d1 =  pd.DataFrame( num_attributes.apply( np.std ) ).T
d2 =  pd.DataFrame( num_attributes.apply( min  ) ).T
d3 = pd.DataFrame( num_attributes.apply( max)).T
d4 =  pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() )).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() )).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() )).T


#concatenar

m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6] ).T.reset_index()
m.columns = ['atributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']

In [ ]:
m


In [ ]:
sns.distplot(df1['competition_open_since_year'])

### 1.7.2 Categorical Attributes

In [ ]:
cat_attributes.apply( lambda x: x.unique().shape[0]) #quantos niveis cada variavel cat tem. 

In [ ]:
sns.boxplot(x = 'state_holiday', y = 'sales', data = df1) #dimensoes diferentes causam graficos muito espacados


In [ ]:
aux1 = df1[(df1['state_holiday'] != '0') & (df1['sales'] > 0)]

plt.subplot(1, 3, 1) #1 linha com 3 graph e esse na 1 posicao
sns.boxplot(x = 'state_holiday', y = 'sales', data = aux1)

plt.subplot(1, 3, 2) #1 linha com 3 graph e esse na 2 posicao
sns.boxplot(x = 'store_type', y = 'sales', data = aux1)

plt.subplot(1, 3, 3) #1 linha com 3 graph e esse na 3 posicao
sns.boxplot(x = 'assortment', y = 'sales', data = aux1)

 - #interpretacao dos boxplt = quartil  75, 50 (mediana) e 25. Comparar medianas.

# Tetse


In [6]:
#panda profile 
import pandas as pd
from pandas_profiling import ProfileReport
import seaborn as sn

In [2]:
df = pd.read_csv( '../datascience_em_producao/data/train.csv', low_memory=False )

In [7]:
df.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday'],
      dtype='object')

In [ ]:
sn.barplot(data = df)

In [ ]:
profile = ProfileReport(df, title = 'train')


In [ ]:
profile.to_notebook_iframe()